## Create a Diallel Table for Origin Selection Step by Step

In [1]:
import pandas as pd
import numpy as np
import requests
import s3fs
import MidParentValue as MP
import rulefilter as rf
import utils
import master_table as mt
import collections
import re
import os
import warnings
warnings.filterwarnings("ignore")
import dask
from dask import delayed
import dask.dataframe as dd
from functools import reduce
import itertools
import time
import json
import hvac
import boto3

/usr/local/anaconda/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/usr/local/anaconda/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/usr/local/anaconda/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
########## Import GCA master file 

if 'ec2' in os.environ['HOME']:
    master_file = utils.read_as_dask_1('ycao1/OriginSelection/master_wide_test.csv', bucket='veg-apd-sdi-predictiveanalytcs-prod-workspace')
else:
    master_file = utils.read_as_dask('ycao1/OriginSelection/master_wide_test.csv', bucket='veg-apd-sdi-predictiveanalytcs-prod-workspace')

In [3]:
master_file = master_file.compute().set_index(pd.Index(range(master_file.compute().shape[0])))

In [4]:
########## Import Key Inbred Lines 

key_inbred = pd.read_csv('/mnt/TempSC/2020_key_inbred_lines_tempSC.csv')

In [5]:
########## Filter by Key Inbred

master_file_filtered = master_file[master_file['PEDIGREE_NAME'].isin(key_inbred['Pedigree'])]

In [6]:
########## Filter by Heterotic Group

master_file_filtered = rf.filter_by_HETGP(master_file_filtered, het=['678', '83', '89', 'A', 'SLM804'])

In [7]:
########### Split the filtered master file by INBRED_MKT_TYPE, TEMP PROC and TEMP FRESH

master_file_proc = master_file_filtered[master_file_filtered['INBRED_MKT_TYPE_GCV'] == 'TEMP PROC']
print(f'{"Master Shape Proc:":20}{master_file_proc.shape}')

master_file_fresh = master_file_filtered[master_file_filtered['INBRED_MKT_TYPE_GCV'] == 'TEMP FRESH']
print(f'{"Master Shape fresh:":20}{master_file_fresh.shape}')

Master Shape Proc:  (107, 144)
Master Shape fresh: (140, 144)


In [8]:
########### Getting Trait list 
# master_file_filtered = master_file
# traits = [col for col in master_file_filtered.columns if '_predicted.value' in col]
traits = [col for col in master_file_filtered.columns if 'predicted.value_' in col]
traits_1 = []
for tt in traits: 
    t_1 = re.sub('predicted.value_', '', tt)
#     if (t_1 != 'SC_RWCT') & (t_1 != 'FBRIX'):
    traits_1.append(t_1)
    

In [9]:
master_file_filtered.shape

(249, 144)

In [10]:
start_1 = time.time()
test = MP.gather_mid_value(dd.from_pandas(master_file_filtered, npartitions=3), traits_1)
print(f'Time for whole data: {time.time()-start_1:.2f} seconds')

Time for whole data: 0.29 seconds


In [ ]:
############# Calculate mid parent value by inbred market type

# Process 
start_1 = time.time()
# diallel_table_1 = MP.gather_mid_value(master_file_filtered, traits_1)
diallel_table_proc = MP.gather_mid_value(dd.from_pandas(master_file_proc, npartitions=3), traits_1)
print(f'Time for Proc: {time.time()-start_1:.2f} seconds')

# Fresh
start_2 = time.time()
# diallel_table_1 = MP.gather_mid_value(master_file_filtered, traits_1)
diallel_table_fresh = MP.gather_mid_value(dd.from_pandas(master_file_fresh, npartitions=3), traits_1)
print(f'Time for Fresh: {time.time()-start_2:.2f} seconds')

In [ ]:
############### Get reference data in one table

reference_cols = [col for col in master_file_filtered.columns if not any(tr in col for tr in traits_1)]
reference_table = master_file_filtered[reference_cols]
reference_table = reference_table.drop(columns=['PEDIGREE'], axis=1)
reference_table = reference_table.drop_duplicates(subset='PEDIGREE_NAME', keep='last') # there are duplicates

In [ ]:
############### Add reference table to diallel table

# Add reference table to diallel table
start = time.time()
print(f'{"Master Shape BEFORE adding reference data:":20}{diallel_table_proc.shape}')
diallel_table_proc = utils.add_reference_dat(diallel_table_proc, reference_table)
print(f'{"Master Shape AFTER adding reference data:":20}{diallel_table_proc.shape}')
diallel_table_fresh = utils.add_reference_dat(diallel_table_fresh, reference_table)
end = time.time()
print(f'Time for Proc: {end-start:.2f} seconds')

In [ ]:
############### Add clean origin to the table
diallel_table_proc['clean_origin'] = diallel_table_proc['P1_M.GERMPLASM.ORIGIN'] + '__' + diallel_table_proc['P2_M.GERMPLASM.ORIGIN']

### Origin Selection: Create New DevX

In [ ]:
############## Filter new dev crosses by het and inbred stage combos

# Proc
het_comb_proc = ['A_A', '678_678', '83_678','678_83']
inbred_stage_comb_proc = ['PS2_PS2', 'PS2_PS3','PS3_PS3', 'PS2_CM', 'PS3_CM','CM_CM', 'CM_P1', 'CM_P2', 'CM_P3',
                         'PS3_PS2', 'CM_PS2', 'CM_PS3', 'P1_CM', 'P2_CM', 'CM_P3']

diallel_table_proc_filtered = rf.filter_on_het_combination(diallel_table_proc, het_comb_proc)
diallel_table_proc_filtered = rf.filter_on_inbrdstg(diallel_table_proc_filtered, inbred_stage_comb_proc)

# Fresh
het_comb_fresh = ['A_A', '678_678', 'SLM804_SLM804', '678_SLM804', '89_SL804', '83_678', 'SLM804_678', 'SL804_89', '678_83']
inbred_stage_comb_fresh = ['PS2_PS2', 'PS2_PS3', 'PS3_PS3', 'PS2_CM', 'PS3_CM','CM_CM', 'CM_P1', 'CM_P2', 'CM_P3',
                          'PS3_PS2', 'CM_PS2', 'CM_PS3', 'P1_CM', 'P2_CM', 'CM_P3']

diallel_table_fresh_filtered = rf.filter_on_het_combination(diallel_table_fresh, het_comb_fresh)
diallel_table_fresh_filtered = rf.filter_on_inbrdstg(diallel_table_fresh_filtered, inbred_stage_comb_fresh)


####################### Eliminate ORIGIN = ORIGIN
diallel_table_proc_filtered = rf.remove_same_origin(diallel_table_proc_filtered)
diallel_table_fresh_filtered = rf.remove_same_origin(diallel_table_fresh_filtered)


####################### Write results to a csv file

diallel_table_proc_filtered.to_csv('/mnt/TempSC/temp_sc_proc_diallel.csv')
diallel_table_fresh_filtered.to_csv('/mnt/TempSC/temp_sc_fresh_diallel.csv')

###################### Upload file to S3

import logging
from botocore.exceptions import ClientError

utils.upload_file(file_name = "/mnt/TempSC/temp_sc_fresh_diallel.csv", bucket="veg-apd-sdi-predictiveanalytcs-prod-workspace",
            object_name = 'ycao1/OriginSelection/Corn/DiallelTable/+OriginSelection/Temp_fresh_diallel.csv')
utils.upload_file(file_name = "/mnt/TempSC//temp_sc_proc_diallel.csv", bucket="veg-apd-sdi-predictiveanalytcs-prod-workspace",
            object_name = 'ycao1/OriginSelection/Corn/DiallelTable/+OriginSelection/Temp_proc_diallel.csv')
